In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder

In [48]:
df = pd.read_csv("Traffic_Crashes_Resulting_in_Injury_20240518.csv")
display(df.iloc[:, 3])
display(df.iloc[:, 15])

C:\Users\emily\AppData\Local\Temp\ipykernel_6692\3174712082.py:1: DtypeWarning: Columns (3,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Traffic_Crashes_Resulting_in_Injury_20240518.csv")


0        180085641
1        220672946
2        130379165
3        220527222
4          4395868
           ...    
59004    230392695
59005    230713386
59006    230347242
59007    230886886
59008    230728169
Name: case_id_pkey, Length: 59009, dtype: object

0        3801
1        3801
2        3801
3        3801
4        3801
         ... 
59004    3801
59005    3801
59006    3801
59007    3801
59008    3801
Name: juris, Length: 59009, dtype: object

In [49]:
df = df.drop(columns=['cnn_intrsctn_fkey', 'cnn_sgmt_fkey', 'case_id_pkey', 
                      'tb_latitude', 'tb_longitude', 'geocode_source', 
                      'officer_id', 'beat_number', 'direction', 
                      'control_device', 'vz_pcf_group', 'vz_pcf_description', 
                      'vz_pcf_link', 'number_killed', 'number_injured', 
                      'street_view', 'dph_col_grp', 'party_at_fault',
                      'party1_dir_of_travel', 'party1_move_pre_acc', 
                      'party2_dir_of_travel', 'party2_move_pre_acc', 
                      'point', 'data_as_of', 'data_updated_at', 
                      'data_loaded_at', 'Neighborhoods', 'SF Find Neighborhoods', 
                      'Current Police Districts', 'Current Supervisor Districts', 'Analysis Neighborhoods',
                      'reporting_district'])
df = df.dropna(axis="index", subset=["collision_time"])

In [50]:

display(df["road_cond_1"].unique())
display(df["road_cond_2"].unique())
df["road_cond_1"].where(df["road_cond_1"] != "Holes, Deep Rut", other="Holes, Deep Ruts", inplace=True)
df["road_cond_1"].where(df["road_cond_1"] != "No Unusual Condition", other="None", inplace=True)
df["road_cond_1"].where(df["road_cond_1"] != "Not Stated", other="None", inplace=True)
df["road_cond_2"].where(df["road_cond_2"] != "No Unusual Condition", other="None", inplace=True)
df["road_cond_2"].where(df["road_cond_2"] != "Not Stated", other="None", inplace=True)
display(df["road_cond_1"].unique())
display(df["road_cond_2"].unique())
df["road_cond_comb"] = df["road_cond_1"] + ";" + df["road_cond_2"]
df["road_cond_comb"] = df["road_cond_comb"].str.split(";")
df = df.drop(columns=['road_cond_1', 'road_cond_2'])
df.loc[:, "juris"] = df.loc[:, "juris"].astype(str)
display(df)

array(['No Unusual Condition', 'Obstruction on Roadway',
       'Holes, Deep Ruts', 'Not Stated', 'Loose Material on Roadway',
       'Other', 'Construction or Repair Zone', 'Holes, Deep Rut',
       'Reduced Roadway Width', 'Flooded'], dtype=object)

array(['Not Stated', 'Other', 'Construction or Repair Zone',
       'Loose Material on Roadway', 'No Unusual Condition',
       'Reduced Roadway Width', 'Flooded', 'Obstruction on Roadway'],
      dtype=object)

array(['None', 'Obstruction on Roadway', 'Holes, Deep Ruts',
       'Loose Material on Roadway', 'Other',
       'Construction or Repair Zone', 'Reduced Roadway Width', 'Flooded'],
      dtype=object)

array(['None', 'Other', 'Construction or Repair Zone',
       'Loose Material on Roadway', 'Reduced Roadway Width', 'Flooded',
       'Obstruction on Roadway'], dtype=object)

,unique_id,geocode_location,collision_datetime,collision_date,collision_time,accident_year,month,day_of_week,time_cat,juris,...,mviw,ped_action,road_surface,lighting,intersection,vz_pcf_code,dph_col_grp_description,party1_type,party2_type,road_cond_comb
0,36434,CITY STREET,01/26/2018 05:45:00 PM,2018 January 26,17:45:00,2018,January,Friday,2:01 pm to 6:00 pm,3801,...,Other Motor Vehicle,Not Stated,Dry,Dark - Street Lights,Intersection <= 20ft,Unknown,Vehicle(s) Only Involved,Driver,Driver,"[None, None]"
1,59922,CITY STREET,10/01/2022 02:27:00 PM,2022 October 01,14:27:00,2022,October,Saturday,2:01 pm to 6:00 pm,3801,...,Bicycle,No Pedestrian Involved,Dry,Daylight,Midblock > 20ft,22350,Vehicle-Bicycle,Bicyclist,Driver,"[None, None]"
2,25680,CITY STREET,05/08/2013 10:51:00 PM,2013 May 08,22:51:00,2013,May,Wednesday,10:01 pm to 2:00 am,3801,...,Pedestrian,Crossing in Crosswalk at Intersection,Dry,Dark - Street Lights,Intersection <= 20ft,Unknown,Vehicle-Pedestrian,Driver,Pedestrian,"[None, None]"
3,58691,CITY STREET,08/07/2022 10:18:00 PM,2022 August 07,22:18:00,2022,August,Sunday,10:01 pm to 2:00 am,3801,...,Fixed Object,No Pedestrian Involved,Dry,Dark - Street Lights,Midblock > 20ft,Unknown,Bicycle Only,Bicyclist,NaN,"[Obstruction on Roadway, Other]"
4,30142,CITY STREET,09/03/2009 01:35:00 PM,2009 September 03,13:35:00,2009,September,Thursday,10:01 am to 2:00 pm,3801,...,Parked Motor Vehicle,No Pedestrian Involved,Dry,Daylight,Intersection Rear End <= 150ft,Unknown,Vehicle(s) Only Involved,Driver,Parked Vehicle,"[None, None]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59004,74610,CITY STREET,06/07/2023 02:10:00 PM,2023 June 07,14:10:00,2023,June,Wednesday,2:01 pm to 6:00 pm,3801,...,Fixed Object,No Pedestrian Involved,Dry,Daylight,Intersection <= 20ft,22350,Vehicle(s) Only Involved,Driver,NaN,"[None, None]"
59005,74892,CITY STREET,10/04/2023 02:00:00 PM,2023 October 04,14:00:00,2023,October,Wednesday,10:01 am to 2:00 pm,3801,...,Other Motor Vehicle,No Pedestrian Involved,Dry,Daylight,Intersection <= 20ft,21453(a),Vehicle(s) Only Involved,Driver,Driver,"[None, None]"
59006,75330,CITY STREET,05/19/2023 09:40:00 PM,2023 May 19,21:40:00,2023,May,Friday,6:01 pm to 10:00 pm,3801,...,Other Motor Vehicle,No Pedestrian Involved,Dry,Dark - Street Lights,Intersection <= 20ft,21801(a),Vehicle(s) Only Involved,Driver,Driver,"[None, None]"
59007,76954,CITY STREET,12/19/2023 08:11:00 PM,2023 December 19,20:11:00,2023,December,Tuesday,6:01 pm to 10:00 pm,3801,...,Motor Vehicle on Other Roadway,No Pedestrian Involved,Dry,Daylight,Intersection <= 20ft,21801(a),Vehicle(s) Only Involved,Driver,Other,"[None, None]"


In [30]:
df.columns

Index(['unique_id', 'geocode_location', 'collision_datetime', 'collision_date',
       'collision_time', 'accident_year', 'month', 'day_of_week', 'time_cat',
       'juris', 'reporting_district', 'primary_rd', 'secondary_rd', 'distance',
       'weather_1', 'weather_2', 'collision_severity', 'type_of_collision',
       'mviw', 'ped_action', 'road_surface', 'lighting', 'intersection',
       'vz_pcf_code', 'dph_col_grp_description', 'party1_type', 'party2_type',
       'road_cond_comb'],
      dtype='object')

In [43]:
df["road_cond_comb"]

0                               [None, None]
1                               [None, None]
2                               [None, None]
3            [Obstruction on Roadway, Other]
4                               [None, None]
                        ...                 
59004                           [None, None]
59005                           [None, None]
59006                           [None, None]
59007                           [None, None]
59008    [Construction or Repair Zone, None]
Name: road_cond_comb, Length: 58950, dtype: object

In [51]:
mlb = MultiLabelBinarizer()
mlb.fit_transform(df["road_cond_comb"])

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [27]:
df.isnull().sum()

unique_id                     0
geocode_location              0
collision_datetime            0
collision_date                0
collision_time                0
accident_year                 0
month                         0
day_of_week                   8
time_cat                      0
juris                         0
reporting_district         6503
primary_rd                    0
secondary_rd                105
distance                     78
weather_1                     0
weather_2                   793
collision_severity            0
type_of_collision             0
mviw                          0
ped_action                    0
road_surface                  0
lighting                      0
intersection                  1
vz_pcf_code                 172
dph_col_grp_description       0
party1_type                  11
party2_type                4215
road_cond_comb                0
dtype: int64

In [28]:
df["reporting_district"].unique()

array(['Bayview', 'Richmond', 'PARK', 'NORTH', 'PARKS', 'E', 'Park', nan,
       'RICHMOND', 'Mission', '3E', 'Southern', 'Traffic', 'Central',
       'Ingleside', 'INGLESIDE', 'Northern', 'NOTHERN', 'SOUTHERN',
       'BAYVIEW', 'Tenderloin', 'MISSION', 'CENTRAL', 'D', 'NORTHERN',
       'COI', 'TARAVAL', 'COJ', 'B', 'TARAV', 'A', 'BAYV', 'INGLE', 'COD',
       'CO B', 'COE', 'Taraval', 'CENTR', 'TENDE', 'COC', '3B', 'G',
       'CO I', 'TENDERLOIN', 'BAY VIEW', 'SOUTHERN S', 'BAYVI', 'PARL',
       '3H15D', '3H15E', 'SOUTH', 'MISSI', 'CONTRAE', 'MISSION ST', 'COB',
       'H', '3E13C', 'TRAFFIC', '3H11E', 'COH', 'COA', 'RICHM', 'RICH',
       '3I16D', 'SANFR', '4B6A', 'F', 'SF', 'ARAVA', 'CO D', 'CO A',
       '4B105', 'SAN FRANCI', 'I', 'C01', 'TENDERLON', 'TARAR', 'C',
       'NORTHER', '3D', 'NORHTHERN', 'TEDERLOIN', 'INGLESIOE',
       'COMPANY B', '3J13D', '3801', '3J', '4B8G', 'CO H', '3C90', 'J',
       'TARA', 'CENTRAK', 'H4', 'COF', '01082', '3A', '901', '07/20/2014',
      